In [1]:
import pandas as pd
import numpy as np

                                    MASUKINNN NAMA FILE EXCEL DATA KARYAWAN LENGKAP                                    

In [3]:
data_mentah_karyawan = pd.read_excel('absensi 2024.xlsx') # MASUKIN NAMA FILE DATA KARYAWAN LENGKAP DISINI
data_mentah_karyawan.drop(columns='No')
data_karyawan = data_mentah_karyawan.dropna(axis=0, how='all')
data_karyawan['NEW MANUAL'] = data_karyawan['NEW MANUAL'].astype(str).str.extract(r'(\d+)', expand=False).astype('Int64')
data_karyawan = data_karyawan.sort_values(by='NEW MANUAL')
data_karyawan = data_karyawan.reset_index(drop=True)
data_karyawan

,No,NAMA KARYAWAN,NEW MANUAL,NEW SISTEM,UNIT KERJA,TMK
0,1,SITI 1,1,SITI - 11,WAKIL REKTOR I,35000
1,2,SITI 2,2,SITI - 12,WAKIL REKTOR II,35000
2,3,SITI 3,3,SITI - 13,WAKIL REKTOR III,35000
3,4,SITI 4,4,SITI - 14,SEKRETARIS REKTOR,35000
4,5,SITI 5,5,SITI - 15,SEKRETARIS REKTOR,35000
...,...,...,...,...,...,...
181,178,SITI 182,182,SITI - 192,STAF PUSTIKOM,0
182,179,SITI 183,183,SITI - 193,PJ. LAB. KEPERAWATAN,0
183,180,SITI 184,184,SITI - 194,STAF RUMGA,0
184,181,SITI 185,185,SITI - 195,NaN,0


                                     MASUKINNN NAMA FILE EXCEL DATA FINGERPRINT MENTAH                                                                               

In [8]:
data_mentah_finger = pd.read_excel("01 jan - 08 jan`24.xls") # MASUKIN NAMA FILE FINGER DISINI #
data_mentah_finger['tanggal'] = pd.to_datetime(data_mentah_finger ['Unnamed: 3'], format='%d/%m/%Y %H:%M')
data_mentah_finger['tanggal'] = data_mentah_finger['tanggal'].dt.date

data_finger = data_mentah_finger.drop(columns='Unnamed: 0')
data_finger['tanggals'] = pd.to_datetime(data_finger['Unnamed: 3'], format='%d/%m/%Y %H:%M')
data_finger['jam'] = data_finger['tanggals'].dt.strftime('%H:%M')

data_finger = data_finger.drop(columns=['tanggals'])
data_finger.drop(columns='Unnamed: 7', inplace=True)
data_finger.drop(columns='Unnamed: 6', inplace=True)
data_finger.rename(columns={'Unnamed: 1': 'NAMA KARYAWAN'}, inplace=True)
data_finger.rename(columns={'Unnamed: 2': 'NEW MANUAL'}, inplace=True)
data_finger.head(10)

,NAMA KARYAWAN,NEW MANUAL,Unnamed: 3,Unnamed: 4,Unnamed: 5,tanggal,jam
0,SITI 1,1,04/01/2024 8:11,C/Masuk,1,2024-01-04,08:11
1,SITI 2,2,04/01/2024 16:45,C/Keluar,1,2024-01-04,16:45
2,SITI 3,3,05/01/2024 8:35,C/Masuk,1,2024-01-05,08:35
3,SITI 4,4,08/01/2024 9:25,C/Masuk,1,2024-01-08,09:25
4,SITI 5,5,03/01/2024 8:57,C/Masuk,1,2024-01-03,08:57
5,SITI 6,6,03/01/2024 9:02,C/Masuk,1,2024-01-03,09:02
6,SITI 7,7,03/01/2024 11:05,C/Masuk,1,2024-01-03,11:05
7,SITI 8,8,03/01/2024 14:21,C/Keluar,1,2024-01-03,14:21
8,SITI 9,9,03/01/2024 17:26,C/Masuk,2,2024-01-03,17:26
9,SITI 10,10,03/01/2024 17:31,C/Keluar,1,2024-01-03,17:31


                                 MASUKINNN TANGGAL FINGER MAU DARI TANGGAL BERAPA SAMPAI TANGGAL BERAPA                   

In [10]:
                        #################   MASUKAN TANGGAL FINGER DISINI  #################

tanggal_finger = pd.date_range(start='2024-01-01', end='2024-01-08', freq='D')
data_finger_hampir_fix = pd.DataFrame(columns=['NEW MANUAL', 'NAMA KARYAWAN'] + tanggal_finger.strftime('%d/%m/%Y').tolist())

start_date_finger = pd.to_datetime('2024-01-01')
end_date_finger = pd.to_datetime('2024-01-08')

data_finger['tanggal'] = pd.to_datetime(data_finger['tanggal'])

for name, group in data_finger.groupby('NEW MANUAL'):
    new_row = {'NEW MANUAL': name, 'NAMA KARYAWAN': group['NAMA KARYAWAN'].iloc[0]}
    
    current_date = start_date_finger
    while current_date <= end_date_finger:
        formatted_date = current_date.strftime('%d/%m/%Y')
        new_row[formatted_date] = "" if ((group['tanggal'].dt.day == current_date.day) & (group['Unnamed: 5'] == 1)).any() else ''
        current_date += pd.DateOffset(days=1)
    
    data_finger_hampir_fix = pd.concat([data_finger_hampir_fix, pd.DataFrame([new_row])], ignore_index=True)

data_finger_hampir_fix = data_finger_hampir_fix.sort_values(by='NEW MANUAL')

for index, row in data_finger.iterrows():
    tanggal = row['tanggal'].strftime('%d/%m/%Y')
    status = row['Unnamed: 4']
    jam = row['jam']

    if status == 'C/Masuk':
        if pd.isna(data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]):
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'Masuk ({jam})'
        else:
            existing_data = data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'{existing_data} masuk ({jam})'
    elif status == 'C/Keluar':
        if pd.isna(data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]):
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'Keluar ({jam})'
        else:
            existing_data = data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal].iloc[0]
            data_finger_hampir_fix.loc[data_finger_hampir_fix['NEW MANUAL'] == row['NEW MANUAL'], tanggal] = f'{existing_data} Keluar ({jam})'

data_finger_fix = data_finger_hampir_fix.fillna('Tidak Masuk')
data_finger_fix = data_finger_fix.applymap(lambda x: 'Tidak Masuk' if x == '' else x)
data_finger_fix

C:\Users\fpamu\AppData\Local\Temp\ipykernel_1224\1434257578.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_finger_fix = data_finger_fix.applymap(lambda x: 'Tidak Masuk' if x == '' else x)


,NEW MANUAL,NAMA KARYAWAN,01/01/2024,02/01/2024,03/01/2024,04/01/2024,05/01/2024,06/01/2024,07/01/2024,08/01/2024
0,1,SITI 1,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (08:11),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
1,2,SITI 2,Tidak Masuk,Tidak Masuk,Tidak Masuk,Keluar (16:45),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
2,3,SITI 3,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (08:35),Tidak Masuk,Tidak Masuk,Tidak Masuk
3,4,SITI 4,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (09:25)
4,5,SITI 5,Tidak Masuk,Tidak Masuk,masuk (08:57),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
...,...,...,...,...,...,...,...,...,...,...
58,59,SITI 59,Tidak Masuk,Tidak Masuk,masuk (08:15),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
59,60,SITI 60,Tidak Masuk,Tidak Masuk,Keluar (16:53),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
60,61,SITI 61,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (06:52),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
61,62,SITI 62,Tidak Masuk,Tidak Masuk,Tidak Masuk,Keluar (16:59),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk


                                                   MERGER DATA FINGER DAN DATA KARYAWAN LENGKAP                       

In [12]:
merged_data = pd.merge(data_karyawan, data_finger_fix, on='NEW MANUAL', how='left', suffixes=('', 'di finger'))
merged_data.drop(columns='NAMA KARYAWANdi finger', inplace=True)
merged_data

,No,NAMA KARYAWAN,NEW MANUAL,NEW SISTEM,UNIT KERJA,TMK,01/01/2024,02/01/2024,03/01/2024,04/01/2024,05/01/2024,06/01/2024,07/01/2024,08/01/2024
0,1,SITI 1,1,SITI - 11,WAKIL REKTOR I,35000,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (08:11),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
1,2,SITI 2,2,SITI - 12,WAKIL REKTOR II,35000,Tidak Masuk,Tidak Masuk,Tidak Masuk,Keluar (16:45),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
2,3,SITI 3,3,SITI - 13,WAKIL REKTOR III,35000,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (08:35),Tidak Masuk,Tidak Masuk,Tidak Masuk
3,4,SITI 4,4,SITI - 14,SEKRETARIS REKTOR,35000,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,masuk (09:25)
4,5,SITI 5,5,SITI - 15,SEKRETARIS REKTOR,35000,Tidak Masuk,Tidak Masuk,masuk (08:57),Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk,Tidak Masuk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,178,SITI 182,182,SITI - 192,STAF PUSTIKOM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,179,SITI 183,183,SITI - 193,PJ. LAB. KEPERAWATAN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,180,SITI 184,184,SITI - 194,STAF RUMGA,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,181,SITI 185,185,SITI - 195,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


                                      INPUT ABSEN MANUAL SURAT TUGAS DAN MASUKAN TANGGAL LAGI                                                                            

In [11]:
data_merger = merged_data

mapping_dict = dict(zip(data_mentah_karyawan['NAMA KARYAWAN'], data_mentah_karyawan.index))

data_merger['Urutan'] = data_merger['NAMA KARYAWAN'].map(mapping_dict)

data_absen_rapih = data_merger.sort_values(by=['Urutan']).drop('Urutan', axis=1).reset_index(drop=True)

data_karyawan_bersih = data_mentah_karyawan.dropna(axis=0, how='all')
data_karyawan_bersih_juga = data_karyawan_bersih.reset_index(drop=True)
data_karyawan_bersih_banget = data_karyawan_bersih_juga[['NEW MANUAL']]

data_karyawan_bersih_fix = data_karyawan_bersih_banget.reset_index(drop=True)
data_absen_rapih['NEW MANUAL'] = data_karyawan_bersih_fix['NEW MANUAL']
data_absen_rapih['NEW MANUAL'].fillna('absen manual', inplace=True)

###################### MASUKIN TANGGAL DIBAWAH #####################
for index, row in data_absen_rapih.iterrows():
    if 'absen manual' not in row['NEW MANUAL'].lower() and row['01/01/2024':'08/01/2024'].isna().all():
        data_absen_rapih.loc[index, '01/01/2024':'08/01/2024'] = 'Tidak Masuk'

        ######## SETELAH DI RUN INPUT MANUAL DI EXCEL, LALU LANJUT UNTUK MEMBACA FILE EXCEL YANG SUDAH DI INPUT, HAPUS TANDA KURUNG DIBAWAH ########

# data_absen_rapih.to_excel("per bulan.xlsx",index=False) # TUTUP LAGI DENGAN TANDA KURUNG JIKA SUDAH DI RUN
# absen_pertanggal_20 = pd.read_excel('per bulan.xlsx.xlsx') # TUTUP LAGI DENGAN TANDA KURUNG JIKA SUDAH DI RUN



NameError: name 'merged_data' is not defined

                                                        MASUKINNN TANGGAL MERAH DI BULAN TERSEBUT                                                 

In [13]:
from datetime import datetime, timedelta

############################################# MASUKAN TANGGAL MERAH  #################################################
tanggal_merah = ["2024-01-09"]
tanggal_columnsk = absen_pertanggal_20.columns[6:]
def hitung_jumlah_ga_masuk(row):
    jumlah_tidak_masuk = 0
    for col in absen_pertanggal_20.columns[6:]: 
        if pd.notna(row[col]) and "Tidak Masuk" in str(row[col]) and pd.to_datetime(col, format="%d/%m/%Y").weekday() < 5 and col not in tanggal_merah:
            jumlah_tidak_masuk += 1

    return jumlah_tidak_masuk
def hitung_sisa_cuti(row):
    jumlah_tidak_masuk = 0
    for col in absen_pertanggal_20.columns[6:-1]:  
        if pd.notna(row[col]) and "Tidak Masuk" in str(row[col]) and pd.to_datetime(col, format="%d/%m/%Y").weekday() < 5 and col not in tanggal_merah:
            jumlah_tidak_masuk += 1

    return 12 - jumlah_tidak_masuk

absen_pertanggal_20["Jumlah Tidak Masuk"] = absen_pertanggal_20.apply(hitung_jumlah_ga_masuk, axis=1)
absen_pertanggal_20["Sisa Cuti"] = absen_pertanggal_20.apply(hitung_sisa_cuti, axis=1)
absen_pertanggal_20['JUMLAH TMK'] = 0

############################################# MASUKAN TANGGAL MERAH  #################################################
tanggal_merah = datetime(2024, 1, 9)

for index, row in absen_pertanggal_20.iterrows():
    for col in absen_pertanggal_20.columns[6:-3]:  
        if pd.notna(row[col]) and isinstance(row[col], str) and 'Tidak Masuk' not in row[col]:
            tanggal = datetime.strptime(col, '%d/%m/%Y')
            if tanggal.weekday() >= 5 or tanggal == tanggal_merah:
                if '(st)' in row[col] or '(pkt)' in row[col]:
                    if row['TMK'] == 35000:
                        absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 70000
                    elif row['TMK'] == 30000:
                       absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 60000
                    else:
                        absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 0
            else:
                if row['TMK'] == 35000:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 35000
                elif row['TMK'] == 30000:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 30000
                elif row['TMK'] == 0:
                    absen_pertanggal_20.at[index, 'JUMLAH TMK'] += 0


total_jumlah_tmk = absen_pertanggal_20['JUMLAH TMK'].sum()
total_df = pd.DataFrame({'NAMA KARYAWAN': ['Total'], 'JUMLAH TMK': [total_jumlah_tmk]})
absen_pertanggal_20= pd.concat([absen_pertanggal_20, total_df], ignore_index=True)
###################### JANGAN LUPA KASIH TANDA KURUNG LAGI JIKA SUDAH DI RUN #################

# absen_pertanggal_20.to_excel('21jan-20feb.xlsx', index= False)
# absen_pertanggal_20.to_excel('21feb-20march.xlsx', index= False)
# absen_pertanggal_20.to_excel('21march-20april.xlsx', index= False)
# absen_pertanggal_20.to_excel('21april-20may.xlsx', index= False)
# absen_pertanggal_20.to_excel('21may-20juni.xlsx', index= False)
# absen_pertanggal_20.to_excel('21juni-20juli.xlsx', index= False)
# absen_pertanggal_20.to_excel('21juli-20agus.xlsx', index= False)
# absen_pertanggal_20.to_excel('21agus-20sept.xlsx', index= False)
# absen_pertanggal_20.to_excel('21sept-20okt.xlsx', index= False)
# absen_pertanggal_20.to_excel('21okt-20nov.xlsx', index= False)
# absen_pertanggal_20.to_excel('21nov-20dec.xlsx', index= False)
# absen_pertanggal_20.to_excel('21dec-20jan.xlsx', index= False)



NameError: name 'absen_pertanggal_20' is not defined